In [1]:
from pycaret.regression import RegressionExperiment
from pycaret.classification import ClassificationExperiment
import pandas as pd
from tqdm import tqdm

In [2]:
data = pd.read_csv('../Datasets/adult.csv')
target_column = 'income'
model_name = 'adult'
exp = ClassificationExperiment()

In [3]:
exp.setup(data, target=target_column, session_id=123)

,Description,Value
0,Session id,123
1,Target,income
2,Target type,Binary
3,Target mapping,"<=50K: 0, >50K: 1"
4,Original data shape,"(48842, 15)"
5,Transformed data shape,"(48842, 67)"
6,Transformed train set shape,"(34189, 67)"
7,Transformed test set shape,"(14653, 67)"
8,Ordinal features,1
9,Numeric features,6


In [6]:
from tqdm.notebook import trange, tqdm
import subprocess
# import sys 
# sys.path.append('/home/ayush/BTP-2/backend/')
# import trainModelAutoML
process = subprocess.Popen(['python3', '/home/ayush/BTP-2/backend/functions/trainModelAutoML.py'], stdout=subprocess.PIPE)

while True:
    output = process.stdout.readline()
    # print(output)
    if output.decode('utf-8') == '' and process.poll() is not None:
        break
    if output:
        print('hello')
        print(output.strip().decode('utf-8'))
        # print("hello")
        # sse.publish(output.strip().decode('utf-8'), type='output')

hello
Pinged your deployment. You successfully connected to MongoDB!
hello
Description              Value
hello
0                    Session id                123
hello
1                        Target             income
hello
2                   Target type             Binary
hello
3                Target mapping  <=50K: 0, >50K: 1
hello
4           Original data shape        (48790, 15)
hello
5        Transformed data shape        (48790, 67)
hello
6   Transformed train set shape        (34153, 67)
hello
7    Transformed test set shape        (14637, 67)
hello
8              Ordinal features                  1
hello
9              Numeric features                  6
hello
10         Categorical features                  8
hello
11                   Preprocess               True
hello
12              Imputation type             simple
hello
13           Numeric imputation               mean
hello
14       Categorical imputation               mode
hello
15     Maximum one-hot encoding  

hello
Model  Accuracy     AUC  Recall   Prec.      F1  \
hello
ada      Ada Boost Classifier    0.8622  0.9171  0.6156  0.7636  0.6815
hello
lr        Logistic Regression    0.8519  0.9063  0.5994  0.7336  0.6596
hello
rf   Random Forest Classifier    0.8550  0.9049  0.6257  0.7305  0.6739
hello
nb                Naive Bayes    0.6843  0.8700  0.9019  0.4254  0.5780
hello
knn    K Neighbors Classifier    0.8253  0.8452  0.5734  0.6544  0.6111
hello
dt   Decision Tree Classifier    0.8159  0.7499  0.6233  0.6141  0.6185
hello
svm       SVM - Linear Kernel    0.8481  0.0000  0.5688  0.7397  0.6408
hello

hello
Kappa     MCC  TT (Sec)
hello
ada  0.5949  0.6007     0.310
hello
lr   0.5662  0.5710     0.611
hello
rf   0.5814  0.5844     0.642
hello
nb   0.3743  0.4422     0.221
hello
knn  0.4991  0.5010     0.656
hello
dt   0.4973  0.4974     0.206
hello
svm  0.5469  0.5559     0.207


In [8]:
print(capturedOutput)

In [5]:
get_leaderboard = exp.get_leaderboard()

In [6]:
print(get_leaderboard.iloc[0]['Model'])

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['age', 'fnlwgt', 'educational-num',
                                             'capital-gain', 'capital-loss',
                                             'hours-per-week'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              co...
                                                              min_samples_leaf=20,
                                                              return_df=True,
                                                              smoothing=10,
                                                              ver

In [11]:

from time import sleep

text = ""
for char in tqdm(["a", "b", "c", "d"]):
    sleep(5)
    text = text + char

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
results = exp.pull()

In [8]:
results

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7984,0.6168,0.2711,0.7081,0.3906,0.2978,0.3479,0.761
knn,K Neighbors Classifier,0.7766,0.6766,0.3221,0.5577,0.4082,0.2825,0.2988,0.782


In [9]:
for column in results.columns:
    print(column)

Model
Accuracy
AUC
Recall
Prec.
F1
Kappa
MCC
TT (Sec)


In [38]:
results[results.index == 'Mean']

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
Mean,0.7984,0.6168,0.2711,0.7081,0.3906,0.2978,0.3479


In [37]:
for metric in results.columns:
    print(results[results.index == 'Mean'][metric].values[0])

0.7984
0.6168
0.2711
0.7081
0.3906
0.2978
0.3479


In [12]:
results.drop(['TT (Sec)','Kappa', 'MCC'], axis=1, inplace=True)

In [13]:
results

,Model,Accuracy,AUC,Recall,Prec.,F1
rf,Random Forest Classifier,0.8569,0.9040,0.6199,0.7401,0.6745
lr,Logistic Regression,0.7984,0.6168,0.2711,0.7081,0.3906


In [21]:
for metric in results.columns:
    # print first row's metric
    if metric == 'Model':
        continue
    print(metric)
    print(results.iloc[0][metric])

Accuracy
0.8569
AUC
0.904
Recall
0.6199
Prec.
0.7401
F1
0.6745
